In [1]:
import numpy as np
from main import *
from copy import deepcopy
from random import choice

In [2]:
info = read_file("sample2.dat")
sp = shortest_path(info["graph"])
sp, info["tasks"]

(array([[  0,  32,  46, ..., 271, 226, 261],
        [ 32,   0,  14, ..., 239, 194, 229],
        [ 46,  14,   0, ..., 253, 208, 243],
        ...,
        [271, 239, 253, ...,   0, 119, 154],
        [226, 194, 208, ..., 119,   0,  35],
        [261, 229, 243, ..., 154,  35,   0]]),
 [((0, 1), 32),
  ((1, 0), 32),
  ((1, 2), 14),
  ((2, 1), 14),
  ((1, 3), 17),
  ((3, 1), 17),
  ((3, 4), 56),
  ((4, 3), 56),
  ((8, 9), 20),
  ((9, 8), 20),
  ((10, 11), 32),
  ((11, 10), 32),
  ((11, 15), 29),
  ((15, 11), 29),
  ((12, 15), 13),
  ((15, 12), 13),
  ((12, 13), 7),
  ((13, 12), 7),
  ((14, 16), 26),
  ((16, 14), 26),
  ((14, 17), 38),
  ((17, 14), 38),
  ((17, 18), 41),
  ((18, 17), 41),
  ((18, 19), 32),
  ((19, 18), 32),
  ((18, 20), 38),
  ((20, 18), 38),
  ((20, 21), 17),
  ((21, 20), 17),
  ((21, 74), 24),
  ((74, 21), 24),
  ((22, 30), 40),
  ((30, 22), 40),
  ((30, 31), 58),
  ((31, 30), 58),
  ((31, 32), 30),
  ((32, 31), 30),
  ((31, 33), 43),
  ((33, 31), 43),
  ((31, 34), 86),

In [3]:
depot, capacity = info["depot"], info["capacity"]
depot, capacity

(0, 305)

In [4]:
def rule1(sp, depot, tasks):
    """maximize the distance from the head of task to the depot"""
    return max(tasks, key=lambda x: sp[depot, x[0][0]])

def rule2(sp, depot, tasks):
    """minimize the distance from the head of task to the depot"""
    return min(tasks, key=lambda x: sp[depot, x[0][0]])

def rule3(sp, tasks):
    """maximize the term dem(t)/sc(t)"""
    return max(tasks, key=lambda x: x[1] / sp[x[0][0], x[0][1]])

def rule4(sp, tasks):
    """minimize the term dem(t)/sc(t)"""
    return min(tasks, key=lambda x: x[1] / sp[x[0][0], x[0][1]])

def rule5(sp, capacity, remain_cap, depot, tasks):
    """use rule1 if the vehicle is less than half full, otherwise use rule2"""
    return rule1(sp, depot, tasks) if remain_cap / capacity > 0.5 else rule2(sp, depot, tasks)

In [5]:
def remove_task(tasks, task):
    tasks.remove(task)
    tasks.remove(((task[0][1], task[0][0]), task[1]))

In [6]:

remain_cap = capacity

routes = [(0, 0)]
cur_vertex = 0  # 车正在哪个顶点
tasks = deepcopy(info["tasks"])
while True:
    valid_tasks = []
    for task in tasks:
        if task[1] <= remain_cap:  # 这个任务比剩余容量小，放得下
            valid_tasks.append(task)
    
    
    if len(valid_tasks) == 0:
        # 当没有有效的任务时，连回depot
        routes.append((0, 0))
        break

    elif len(valid_tasks) == 2:  # 因为一个tasks有两条记录
        # 只有一个有效任务，最短路径过去
        task = valid_tasks[0] if valid_tasks[0][0][0] < valid_tasks[1][0][0] else valid_tasks[1]
        pos = task[0]
        routes.append(pos)
        cur_vertex = pos[0]  # 修改车的位置
        
        # 移除task
        remove_task(tasks, task)
        # 减少容量
        remain_cap -= valid_tasks[0][1]

        # 一定是最后一个了
        routes.append((0, 0))
        break
    
    else:
        # 有多个任务满足条件，选一个距离最短的
        min_cost = min(valid_tasks, key=lambda x: sp[cur_vertex, x[0][0]])[1]

        # 检查是否有多个距离最短的点
        min_cost_tasks = [t for t in valid_tasks if t[1] == min_cost]
        # print(min_cost_tasks)
        if len(min_cost_tasks) == 1:  # 1，只有一个
            task = min_cost_tasks[0]
        else:  # 多个
            # TODO 使用五种Rules
            # task = choice(min_cost_tasks)
            t1 = rule1(sp, depot, min_cost_tasks)
            t2 = rule2(sp, depot, min_cost_tasks)
            t3 = rule3(sp, min_cost_tasks)
            t4 = rule4(sp, min_cost_tasks)
            t5 = rule5(sp, capacity, remain_cap, depot, min_cost_tasks)
        
        pos = task[0]
        routes.append(pos)
        cur_vertex = pos[0]
        
        # 移除task
        remove_task(tasks, task)
        remain_cap -= min_cost

routes

[(0, 0),
 (58, 43),
 (50, 20),
 (75, 19),
 (74, 22),
 (68, 3),
 (68, 58),
 (68, 57),
 (67, 65),
 (65, 61),
 (64, 62),
 (62, 61),
 (61, 59),
 (0, 0)]

In [9]:
def add_routes(routes, pos):
    """"""
    if len(pos) == 2:  # 单个坐标
        for r in routes:
            r.append(pos)
    else:
        for i in range(len(pos)):
            routes[i].append(pos[i])

In [ ]:
num_rule = 5

remain_cap = capacity

routes = [[(0, 0)] for _ in range(5)]
cur_vertex = 0  # 车正在哪个顶点
tasks = deepcopy(info["tasks"])
while True:
    valid_tasks = []
    for task in tasks:
        if task[1] <= remain_cap:  # 这个任务比剩余容量小，放得下
            valid_tasks.append(task)
    
    
    if len(valid_tasks) == 0:
        # 当没有有效的任务时，连回depot
        add_routes(routes, (0, 0))
        break

    elif len(valid_tasks) == 2:  # 因为一个tasks有两条记录
        # 只有一个有效任务，最短路径过去
        task = valid_tasks[0] if valid_tasks[0][0][0] < valid_tasks[1][0][0] else valid_tasks[1]
        pos = task[0]
        routes.append(pos)
        cur_vertex = pos[0]  # 修改车的位置
        
        # 移除task
        remove_task(tasks, task)
        # 减少容量
        remain_cap -= valid_tasks[0][1]

        # 一定是最后一个了
        add_routes(routes, (0, 0))
        break
    
    else:
        # 有多个任务满足条件，选一个距离最短的
        min_cost = min(valid_tasks, key=lambda x: sp[cur_vertex, x[0][0]])[1]

        # 检查是否有多个距离最短的点
        min_cost_tasks = [t for t in valid_tasks if t[1] == min_cost]
        # print(min_cost_tasks)
        if len(min_cost_tasks) == 1:  # 1，只有一个
            task = min_cost_tasks[0]
            add_routes(routes, task[0])
        else:  # 多个
            # TODO 使用五种Rules
            # task = choice(min_cost_tasks)
            t1 = rule1(sp, depot, min_cost_tasks)
            t2 = rule2(sp, depot, min_cost_tasks)
            t3 = rule3(sp, min_cost_tasks)
            t4 = rule4(sp, min_cost_tasks)
            t5 = rule5(sp, capacity, remain_cap, depot, min_cost_tasks)
            add_routes(routes, [t1[0], t2[0], t3[0], t4[0], t5[0]])
        
        pos = task[0]
        routes.append(pos)
        cur_vertex = pos[0]
        
        # 移除task
        remove_task(tasks, task)
        remain_cap -= min_cost

routes